In [88]:
import os

In [89]:
%pwd

'D:\\exercises\\Cold_Email_Generator\\MLOPS\\End-to-End-Machine-Learning-Project-with-MLFLOW'

In [90]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_files : Path
    unzip_dir : Path



In [91]:
from ML_Project.constants import *
from ML_Project.utils.common import read_yaml, create_directories



In [108]:
class ConfigurationManager:
    def __init__(self,
                 config_path = Config_File_Path,
                 params_path = Params_File_Path,
                 schema_path = Schema_File_Path):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)
        create_directories([self.config['artifacts_root']])
   

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_files = config.local_data_files,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config
    
    

                 
    
        

In [109]:
import os
import zipfile
import requests
from ML_Project import logger
from ML_Project.utils.common import get_size



In [114]:
class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_files):
            response = requests.get(self.config.source_url)
            response.raise_for_status()
            with open(self.config.local_data_files, 'wb') as f:
                f.write(response.content)
            logger.info(f"Downloading file ")
        else:
            logger.info(f"Data already exists with size {get_size(self.config.local_data_files)}")

    def unzip_data(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_files, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    


In [115]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()
except Exception as e:
    print(e)

[2024-12-25 23:13:00,199: INFO: common : Yaml file loadded successfully from config\config.yaml]
[2024-12-25 23:13:00,202: INFO: common : Yaml file loadded successfully from params.yaml]
[2024-12-25 23:13:00,206: INFO: common : Yaml file loadded successfully from schema.yaml]
[2024-12-25 23:13:00,207: INFO: common : Directory created at artifacts]
[2024-12-25 23:13:00,207: INFO: common : Directory created at artifacts/data_ingestion]
[2024-12-25 23:13:00,573: INFO: 306392633 : Downloading file ]
